In [1]:
#!/usr/bin/env python

def get_input(fname):
    coordinates = []
    with open(fname) as f:
        for line in f.readlines():
            parts = line.strip().split(", ")
            lft = parts[0].split("=")
            rgt = parts[1].split("=")
            if lft[0] == "x":
                coordinates.append(([int(lft[1]), int(lft[1])], list(map(int, rgt[1].split("..")))))
            else:
                coordinates.append((list(map(int, rgt[1].split(".."))), [int(lft[1]), int(lft[1])]))
    return coordinates

In [2]:
test_input = get_input("test.txt")

In [3]:
class Map(object):
    SPRING = "+"
    CLAY = "#"
    EMPTY = "."
    SAND = "|"
    WATER = "~"
    grid = None
    minx = 0
    maxx = 0
    miny = 0
    maxy = 0
    def __init__(self, coordinates):
        self.minx = coordinates[0][0][0]
        self.miny = coordinates[0][1][0]
        for coord in coordinates:
            self.minx = min(self.minx, coord[0][0])
            self.maxx = max(self.maxx, coord[0][1])
            self.miny = min(self.miny, coord[1][0])
            self.maxy = max(self.maxy, coord[1][1])
        self.minx -= 1
        self.maxx += 1
        self.grid = [[Map.EMPTY for _ in range(self.maxx - self.minx + 1)] for _ in range(0, self.maxy + 1)]
        for coord in coordinates:
            # translation
            coord[0][0] -= self.minx
            coord[0][1] -= self.minx
            for i in range(coord[1][0], 1 + coord[1][1]):
                for j in range(coord[0][0], 1 + coord[0][1]):
                    self.grid[i][j] = Map.CLAY
        self.grid[0][500 - self.minx] = Map.SPRING
    def flow(self):
        path = [(0, 500 - self.minx)]
        loops = 0
        while len(path) > 0:
            loops += 1
            head = path[-1]
            if head[0] < self.maxy:
                # down?
                if self.grid[head[0] + 1][head[1]] == Map.EMPTY:
                    path.append((head[0] + 1, head[1]))
                    self.grid[head[0] + 1][head[1]] = Map.SAND
                elif self.grid[head[0] + 1][head[1]] in (Map.CLAY, Map.WATER):
                    # fill?
                    fill = [False, False]
                    lft = head[1]
                    rgt = head[1]
                    while lft >= 0:
                        if self.grid[head[0] + 1][lft - 1] not in (Map.CLAY, Map.WATER):
                            break
                        elif self.grid[head[0]][lft - 1] in (Map.CLAY, Map.WATER):
                            fill[0] = True
                            break
                        lft -= 1
                    while rgt <= self.maxx - self.minx:
                        if self.grid[head[0] + 1][rgt + 1] not in (Map.CLAY, Map.WATER):
                            break
                        elif self.grid[head[0]][rgt + 1] in (Map.CLAY, Map.WATER):
                            fill[1] = True
                            break
                        rgt += 1
                    if all(fill):
                        for i in range(lft, 1 + rgt):
                            self.grid[head[0]][i] = Map.WATER
                        path.pop()
                        continue
                    elif head[1] > 0 and self.grid[head[0]][head[1] - 1] == Map.EMPTY:
                        path.append((head[0], head[1] - 1))
                        self.grid[head[0]][head[1] - 1] = Map.SAND
                    elif head[1] <= self.maxx - self.minx and self.grid[head[0]][head[1] + 1] == Map.EMPTY:
                        path.append((head[0], head[1] + 1))
                        self.grid[head[0]][head[1] + 1] = Map.SAND
                    else:
                        path.pop()
                else:
                    path.pop()
            else:
                path.pop()
    def __str__(self):
        return "\n".join(''.join(line) for line in self.grid)
    def __repr__(self):
        return str(self)

In [4]:
test_map = Map(get_input("test.txt"))

In [5]:
test_map.flow()

In [6]:
print(sum(1 for i in range(len(test_map.grid)) for j in range(len(test_map.grid[i])) if test_map.grid[i][j] in (Map.WATER, Map.SAND)) - test_map.miny + 1)

57


In [7]:
actual_map = Map(get_input("input.txt"))

In [8]:
actual_map.flow()

In [9]:
print(sum(1 for i in range(len(actual_map.grid)) for j in range(len(actual_map.grid[i])) if actual_map.grid[i][j] in (Map.WATER, Map.SAND)) - actual_map.miny + 1)

39162


In [10]:
print(sum(1 for i in range(len(test_map.grid)) for j in range(len(test_map.grid[i])) if test_map.grid[i][j] in (Map.WATER)))

29


In [11]:
print(sum(1 for i in range(len(actual_map.grid)) for j in range(len(actual_map.grid[i])) if actual_map.grid[i][j] in (Map.WATER)))

32047
